### Importing all relevant libraries

In [ ]:
%matplotlib inline
from Functions import A1, A2
from  itertools import combinations
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the functions

In [ ]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])

run = 50

p = 13

functions = [A1, A2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

df3 = pd.read_csv('2019.03.05_scrambled.csv').T.iloc[1:].T

df4 = pd.DataFrame(sobol_seq.i4_sobol_generate(2*k, run*(2**p+5*2**(p-4))))

df = pd.read_csv('2019.04.24.index.csv',index_col=0)

df5 = df4.reindex(df.index).reset_index(drop=True)

### Running the algorithm

In [17]:
n = 2

p = 13

for pi in range(5,8):
    run_samples = []

    MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    for r in range (run):
        run_samples.append(df5.iloc[r*(2**pi+5*2**(pi-4)):(r+1)*(2**pi+5*2**(pi-4))].reset_index(drop=True))
        run_samples[-1] = run_samples[-1].T.iloc[df3.iloc[r]].reset_index(drop=True).T

        sample_Matrices = [run_samples[-1].iloc[:,m*int(len(run_samples[-1].T)/n):(m+1)*int(len(run_samples[-1].T)/n)] 
                           for m in range(n)]

        mixed_Matrices = []
        for j in range(k):
            mixed_Matrices.append(sample_Matrices[0].copy())
            mixed_Matrices[-1][j]=sample_Matrices[-1][k+j]

        for f in functions:
            Est_cm1 = pd.DataFrame(index=[j for j in reversed(range(k))])

            E_dic = {j:[] for j in range(k)}

            E_list = []

            c1 = [0]

            for qa in range(2):

                if qa == 0:
                    c1.append(2**(pi-2))

                    qample = sample_Matrices[0].iloc[sum(c1[:-1]):sum(c1)]

                    for j in range(k):
                        E_dic[j] = (0.5*(f(qample)-
                                         f(mixed_Matrices[j].iloc[qample.index]))**2).values

                    E_df = pd.DataFrame.from_dict(E_dic,orient='index').T

                    E_list.append(E_df)

                    Est_cm = np.std(E_df)

                    Est_cm2 = Est_cm.sort_values(ascending=False).iloc[:-2]

                else:
                    e_li1 = []
                    e_li = []

                    for isM, risM in sample_Matrices[0].iloc[sum(c1):2**pi].iterrows():
                        e_dic = {ij:[] for ij in Est_cm2.index}
                        Est_cm3 = Est_cm2.append(Est_cm2[np.random.choice(Est_cm2.index, k-len(Est_cm2),replace=False)])

                        E_l_dic=e_dic.copy()  

                        for ij in Est_cm3.index[:-2]:
                            E_l_dic[ij] = f(risM)
                            E_l_dic[ij] = np.append(E_l_dic[ij],f(mixed_Matrices[ij].iloc[isM]))

                        for ii in range(len(Est_cm2.index),k):    
                            ray = risM.copy()
                            ray[Est_cm3.index[ii]] = np.random.rand()

                            E_l_dic[Est_cm3.index[ii]] = np.append(E_l_dic[Est_cm3.index[ii]],(f(ray)))

                        for ii,ij in enumerate(Est_cm3.index):
                            cc = list(combinations(E_l_dic[ij],2))
                            e_dic[ij] = [0.5*(c[1]-c[0])**2 for c in cc]

                        e_li.append(pd.DataFrame.from_dict(e_dic,orient='index').T)
                        
                        e_li1.append(pd.concat([E_df,pd.concat(e_li)]).expanding(1).mean().tail(1))
                        
            me2 = pd.concat([E_df.expanding(1).mean(),pd.concat(e_li1)]).reset_index(drop=True)

            Var = f(sample_Matrices[0].iloc[:2**pi]).expanding(1).var(ddof=0)

            T = (me2.T/Var).T

            AE_v = abs(T-AE_df.loc[f.__name__])
            MAE_r = AE_v.mean(axis=1)
            MAE_r.index=(MAE_r.index+1)*(k+1)
            MAE_dic[f.__name__][r]=MAE_r.dropna()

    writer = pd.ExcelWriter(str(2**pi)+' New_Estimator_B_config.xlsx', engine='xlsxwriter')

    for mk in MAE_dic.keys():
        MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)

    writer.save()